In [11]:
import os
import sys
import pandas as pd
import numpy as np
import json
import json

qpath='F:/TREC2017/Analysis'
os.chdir(qpath)
os.getcwd()
#path1='F:/TREC2017/Analysis/Ranklist/jm/fin'
path1='F:/TREC2017/Analysis/Ranklist/jm112'
path1='F:/TREC2017/Analysis/Ranklist/diri'
#path1='F:/TREC2017/Analysis/Ranklist/bm25'


In [12]:
ip=[]
with open('tid95-2.txt','r') as fin:
    line= fin.readlines()
    for l in line:
        l=l.strip().split(' ')
        #tpn=l[0]
        tpid=l[1]
        ip.append(tpid)      
print len(ip)

95


In [13]:
import pandas as pd
import numpy as np


qrels = pd.read_csv('Qrel/rts2017-batch-qrels.txt', sep=' ',header=None)
#runfile = pd.read_csv('merge_B_Jelinek0.2_0.6_normal_0.4_21_15_final_luchen.txt', sep=' ')
epoch = pd.read_csv('Qrel/rts2017-batch-tweets2dayepoch.txt', sep=' ',header=None)
epoch.columns=['tweet_id','date','time']
qrels.columns=['profile','qo','tweet_id','relevance']
qrels.drop(qrels[[1]],axis=1,inplace=True)
cluster = pd.read_csv('Qrel/clusters.txt', sep=' ',header=None)
cluster.columns=['profile','cluster','tweet_id']

#print qrels.head() #print runfile.head() print cluster.head() print epoch.head()
#cluster.drop(cluster[[3]],axis=1,inplace=True)
#cluster.drop(cluster[[3]],axis=1,inplace=True)
#cluster.drop(cluster[[3]],axis=1,inplace=True)
#cluster.info()
#epoch.drop(epoch[[2]],axis=1,inplace=True)

epoch.info()
print "number of unique:",epoch['tweet_id'].nunique()
#print "number of unique:",epoch['tweet_id','date'].nunique()
#print "number of duplicated:",epoch[epoch.duplicated(['tweet_id','date'])].count()
#epoch=epoch.drop_duplicates(subset=['tweet_id','date'])
#fqrels = pd.read_csv('Qrel_formated.txt', sep=' ',header=None)
#fqrels.columns=['tweet_id','profile','text']
#fqrels=fqrels.drop_duplicates(subset=['tweet_id'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89281 entries, 0 to 89280
Data columns (total 3 columns):
tweet_id    89281 non-null int64
date        89281 non-null int64
time        89281 non-null int64
dtypes: int64(3)
memory usage: 2.0 MB
number of unique: 89281


In [14]:
os.chdir(path1)
#ip=['RTS47']
for i,p in enumerate(ip):
    qn=qrels[(qrels['relevance'] > 0) & (qrels['profile'] == p)]
    #qn.count()


    ij=qn.merge(epoch,how='inner',on=['tweet_id'])
    #ij.head()
    ijc= pd.merge(ij,cluster,how='inner',on=['tweet_id','profile'])
    ijc=ijc.sort_values(by=['date','time'],ascending=[True,True])
    
    os.chdir(path1)
#os.mkdir(p)
    path2=path1 +"/" + p
    os.chdir(path2)

    fn = p + '_qrel.txt'
    #ijc=ijc.sort_values(ijc.columns[3],ascending=[True])
    ijc.to_csv(fn,index=False,sep=' ',header=None)
    #textstring=  p +'text.txt'
    #tqrel.to_csv(textstring,index=False,sep= ' ')
    
    fnstr= p + '_qrel.txt'
    fnstr1 = p + '.txt'
    gq = pd.read_csv(fnstr, sep=' ')
    #gq.columns
#gq.info()
    pt=pd.read_csv(fnstr1,sep=',',header=None)
    pt.columns=['date','profile','qo','tweet_id','rank','score','text']
    
    pt.drop(pt[[2]],axis=1,inplace=True)
    pt1=pt.drop_duplicates(subset='tweet_id')
    pt1.reset_index(drop=True)
    mt=ijc.merge(pt1,how='inner',on=['tweet_id','profile','date'])
#mt.info()
#mt.head()
#mt=mt.sort_values(mt.columns[3],mt.columns[5],ascending=[True,True])
    mt=mt.sort_values(by=['date','rank'],ascending=True)
    mt.to_csv(p +'_qrelinrank.txt',index=False,sep=' ',header=None)
    pt11=pt1['tweet_id'].tolist()
    gq1=ijc['tweet_id'].tolist()
    
    ngq=list(set(gq1)-set(pt11))
    if len(ngq) > 0 :
        ngq_df = pd.DataFrame(ngq)
        ngq_df.columns=['tweet_id']
        qrelmissRank_df= pd.merge(ngq_df,ijc,how='inner',on=['tweet_id'])
        qrelmissRank_df=qrelmissRank_df.sort_values(by=['date'],ascending=[True])
#qrelmissRank_df= pd.merge(qrelmissRank_df,epoch,how='inner',on=['tweet_id'])
        #nqrelmissRank_df= pd.merge(ngq_df,ijc,how='inner',on=['tweet_id'])
        qrelmissRank_df.to_csv(p + '_qrelmissrank.txt',index=False,sep= ' ',header=None)
    #path_rn='C:/Users/lenovo/Google Drive/PHD share folder/paper/trec experiment/rishabh/11417'

    #os.chdir(path_rn)
    os.chdir(path1)
    #run_df=pd.read_csv("C:/Users/lenovo/Google Drive/PHD share folder/trec-rts-2017/runfile/jm6112017_0.1 27.0_20.0_+4+3final_doc_l_grid_ndcg1_again_list.txt",sep=' ',header=None)
#run_df.drop(run_df[[7]],axis=1,inplace=True)
    run_df=pd.read_csv("F:/TREC2017/Analysis/Result/dirichlet/grid/dirichlet_gold_0.1 6.0_3.0.txt",sep=' ',header=None)
    #run_df=pd.read_csv("F:/TREC2017/Analysis/Result/jm112/grid/jm_gold1122017_0.1 21.0_10.0.txt",sep=' ',header=None)

    run_df.drop(run_df[[6]],axis=1,inplace=True)

    run_df.columns=['date','profile','q0','tweet_id','rank','score']
    run_df.drop(run_df[[2]],axis=1,inplace=True)


    run_df=run_df[(run_df['profile'] == p)]
    #run_df.info()
    run_merge_df =pd.merge(mt,run_df,how='inner',on=['tweet_id','profile','date'])
    
    os.chdir(path1)
    os.chdir(p)
    run_merge_df=run_merge_df.sort_values(by=['date','rank_y'],ascending=[True,True])
    run_merge_df.to_csv(p+"_System_summary.txt",index=False,sep=' ')
    
    Qrel_notinsumm_df =  pd.DataFrame(list(set(mt['tweet_id'].tolist()) - set(run_df['tweet_id'].tolist())))
    if(Qrel_notinsumm_df.empty==False):
        Qrel_notinsumm_df.columns=["tweet_id"]
        Qrel_notinsumm_df = pd.merge(Qrel_notinsumm_df,mt,how='inner',on=['tweet_id'])
        Qrel_notinsumm_df.info()
#run_merge_df.info()
        Qrel_notinsumm_df=Qrel_notinsumm_df.sort_values(by=['date','rank'],ascending=[True,True])
        Qrel_notinsumm_df.to_csv(p +'System_notincluded.txt',index=False,sep=' ')
    
    #if not (wt_summ_df.empty):
    wt_summ_df =  pd.DataFrame(list(set(run_df['tweet_id'].tolist()) - set(mt['tweet_id'].tolist())))
    #wt_summ_df.info()
    if(wt_summ_df.empty==False):
        wt_summ_df.columns=["tweet_id"]
        wt_summ_df = pd.merge(wt_summ_df,pt1,how='inner',on='tweet_id')
    #wt_summ_df.head()
#wt_summ_df.info()
        wt_summ_df=wt_summ_df.sort_values(by=['date','rank'],ascending=[True,True])
        wt_summ_df.to_csv(p + "NonRelevantTweetinsummary.txt",index=False,sep= ' ')
    os.chdir(path1)
    if(run_merge_df.empty==False):
        a=run_merge_df['cluster'].nunique()
    else:
        a=0
    b=len(run_merge_df)

    cls = cluster[(cluster['profile'] == p)]
    if(cls.empty==False):
        c= cls['cluster'].nunique()
    else:
        c=0
    if(wt_summ_df.empty==False):
        nrt = len(wt_summ_df)
    else:
        nrt=0
    rt = b-a
    statstr = p + " " + str(c) + " " + str(a) + " " + str (rt) + " " + str(nrt)+"\n"
    #os.chdir(os.pardir)
    fstat = open('stats.txt',"a+")
    fstat.write(statstr)
    fstat.close()




<class 'pandas.core.frame.DataFrame'>
Int64Index: 178 entries, 0 to 177
Data columns (total 9 columns):
tweet_id     178 non-null int64
profile      178 non-null object
relevance    178 non-null int64
date         178 non-null int64
time         178 non-null int64
cluster      178 non-null int64
rank         178 non-null int64
score        178 non-null float64
text         178 non-null object
dtypes: float64(1), int64(6), object(2)
memory usage: 13.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 20
Data columns (total 9 columns):
tweet_id     21 non-null int64
profile      21 non-null object
relevance    21 non-null int64
date         21 non-null int64
time         21 non-null int64
cluster      21 non-null int64
rank         21 non-null int64
score        21 non-null float64
text         21 non-null object
dtypes: float64(1), int64(6), object(2)
memory usage: 1.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Data columns (total 9 colu

In [21]:

b

18

In [31]:
tidall=[]
tidw=[]
day=[20170729,20170730,20170731,20170801,20170802,20170803,20170804,20170805]
d=day[7]
#ip=['RTS47','RTS48','RTS49']
for p in ip:
    os.chdir(path1)
    #os.chdir(p)
    if os.path.exists(path1+'/'+p+'/'+ p+'_qrelmissrank.txt'):
        df=pd.read_csv(p+'/'+ p+'_qrelmissrank.txt',header=None,sep=' ')
        df.columns=['tweet_id','profile','relevaance','date']
        df=df[df['date']==d]
        dtid=list(df['tweet_id'])
        tidall.append(dtid)  
for i in tidall:
    
    for j in i:
        tidw.append(j)

os.chdir(path1)

sday=str(d)
sday=sday[6:8]
with open(sday+"missingtid.txt", "w") as output:
    for t in tidw:
        #print t
        output.write(str(t)+ '\n')
output.close()

   

In [22]:
len(tidall[1])

0

In [14]:
tidall=[]
tidw=[]
day=[20170729,20170730,20170731,20170801,20170802,20170803,20170804,20170805]
d=day[0]
#ip=['RTS47','RTS48','RTS49']
for p in ip:
    os.chdir(path1)
    #os.chdir(p)
    if os.path.exists(path1+'/'+p+'/'+ p+'_qrelinrank.txt'):

    df=pd.read_csv(p+'/'+ p+'_qrelinrank.txt',header=None,sep=' ')
    df.columns=['tweet_id','profile','relevaance','date','rank','scoe','text']
    #df=df[df['date']==d]
    dtid=list(df['tweet_id'])
    tidall.append(dtid)  

ValueError: No columns to parse from file

In [11]:
len(tidall)

39

In [114]:

sday

'30'

In [17]:
len(ip)

95

In [105]:
len(tidw)

6084

In [106]:
os.chdir(path1)


with open("missingtid.txt", "a") as output:
    for t in tidw:
        #print t
        output.write(str(t)+ '\n')
output.close()
